In [40]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import classification_report
from models import create_fixed_input_shape_model, create_variable_input_shape_model
from PIL import Image

In [41]:
IMAGE_SIZE = (160, 160)
NUM_CLASSES = 10

test_dir = "datasets/imagenette2/val"
model_dir = "models"

In [42]:
fixed_model = create_fixed_input_shape_model(IMAGE_SIZE, NUM_CLASSES)
fixed_model.load_weights(os.path.join(model_dir, "fixed-imagenette2.h5"))

In [43]:
variable_model = create_variable_input_shape_model(NUM_CLASSES)
variable_model.load_weights(os.path.join(model_dir, "variable-imagenette2.h5"))

In [44]:
CLASSES = {
    0: 'n01440764',
    1: 'n02102040',
    2: 'n02979186',
    3: 'n03000684',
    4: 'n03028079',
    5: 'n03394916',
    6: 'n03417042',
    7: 'n03425413',
    8: 'n03445777',
    9: 'n03888257'
}

In [45]:
def _load_img(img_path, max_dimension=IMAGE_SIZE[0]):
    #load image from path and convert to array
    img = load_img(img_path, color_mode='rgb', interpolation='nearest', target_size=None)
    
    #downsample image if above allowed size if specified
    img_max_dimension = max(img.size) 
    if img_max_dimension > max_dimension:
        new_dimension = tuple(dim * max_dimension // img_max_dimension for dim in img.size)
        img = img.resize(size=new_dimension, resample=Image.BILINEAR)
        
    return img_to_array(img) # convert PIL image to numpy array

def _pad_img(img, shape):
    #pad images to match largest image in batch
    width, height = shape
    img_width, img_height, _ = img.shape

    diff_w = ((width - img_width) // 2, ((width - img_width) // 2) + ((width - img_width) % 2))
    diff_h = ((height - img_height) // 2, ((height - img_height) // 2) + ((height - img_height) % 2))

    return np.pad(img, (diff_w, diff_h, (0, 0)), mode='constant', constant_values=0.)

def predict(path, model, how, image_size=None, classes=CLASSES):
    if how == "fixed":
        image = load_img(path, color_mode="rgb", target_size=image_size)
        image = img_to_array(image)
    elif how == "variable":
        image = _load_img(path)
        image = _pad_img(image, IMAGE_SIZE)

    image = preprocess_input(image)
    image = np.expand_dims(image, axis=0)

    y_pred = model.predict(image)
    predicted = classes.get(y_pred.argmax())

    return predicted, y_pred.max()

In [46]:
test_df = pd.DataFrame(columns=["image", "class"])
for class_ in CLASSES.values():
    images = [os.path.join(test_dir, class_, image) for image in os.listdir(os.path.join(test_dir, class_))]
    class_df = pd.DataFrame(data={"image": images, "class": class_})
    test_df = test_df.append(class_df, ignore_index=True)

In [47]:
test_df['fixed_pred'] = np.nan
test_df['fixed_conf'] = np.nan
test_df['variable_pred'] = np.nan
test_df['variable_conf'] = np.nan

for idx in test_df.index:
    row = test_df.loc[idx]
    fixed_pred, fixed_conf = predict(row['image'], model=fixed_model, how="fixed", image_size=IMAGE_SIZE)
    variable_pred, variable_conf = predict(row['image'], model=variable_model, how="variable",  image_size=None)

    row['fixed_pred'] = fixed_pred
    row['fixed_conf'] = fixed_conf
    row['variable_pred'] = variable_pred
    row['variable_conf'] = variable_conf

    test_df.loc[idx] = row

In [48]:
print(classification_report(test_df['class'], test_df['fixed_pred'], digits=4))

              precision    recall  f1-score   support

   n01440764     0.9742    0.9767    0.9755       387
   n02102040     0.9871    0.9722    0.9796       395
   n02979186     0.9396    0.9580    0.9487       357
   n03000684     0.9247    0.9223    0.9235       386
   n03028079     0.9849    0.9584    0.9715       409
   n03394916     0.9839    0.9289    0.9556       394
   n03417042     0.9711    0.9512    0.9610       389
   n03425413     0.8882    0.9666    0.9257       419
   n03445777     0.9872    0.9649    0.9759       399
   n03888257     0.9552    0.9846    0.9697       390

    accuracy                         0.9585      3925
   macro avg     0.9596    0.9584    0.9587      3925
weighted avg     0.9595    0.9585    0.9587      3925



In [49]:
print(classification_report(test_df['class'], test_df['variable_pred'], digits=4))

              precision    recall  f1-score   support

   n01440764     0.9512    0.9561    0.9536       387
   n02102040     0.9816    0.9443    0.9626       395
   n02979186     0.9680    0.9328    0.9501       357
   n03000684     0.8889    0.8705    0.8796       386
   n03028079     0.9151    0.9756    0.9444       409
   n03394916     0.9046    0.9391    0.9215       394
   n03417042     0.9289    0.9075    0.9181       389
   n03425413     0.9111    0.9045    0.9078       419
   n03445777     0.9772    0.9649    0.9710       399
   n03888257     0.9298    0.9513    0.9404       390

    accuracy                         0.9348      3925
   macro avg     0.9356    0.9346    0.9349      3925
weighted avg     0.9352    0.9348    0.9348      3925

